In [53]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, ForeignKey
from sqlalchemy.orm import sessionmaker, aliased, relationship
from sqlalchemy import tuple_, and_, or_

# Checking the version

In [3]:
sqlalchemy.__version__

'1.4.41'

# Connecting to the in-memory-only SQLite database

In [4]:
engine = create_engine('sqlite:///:memory:', echo=True)

The echo flag is a shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard logging module.

# Declaraing a Mapping

In [5]:
Base = declarative_base()

In [6]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return f"<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})"

In [7]:
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [8]:
Base.metadata.create_all(engine)

2022-09-14 02:11:23,768 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 02:11:23,768 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-09-14 02:11:23,784 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-14 02:11:23,784 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-09-14 02:11:23,789 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-14 02:11:23,793 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-14 02:11:23,794 INFO sqlalchemy.engine.Engine [no key 0.00174s] ()
2022-09-14 02:11:23,797 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

In [10]:
ed_user.name

'ed'

In [11]:
ed_user.nickname

'edsnickname'

In [12]:
str(ed_user.id)

'None'

# Creating Sessions

In [13]:
Session = sessionmaker(bind=engine)

If the sessions does not have an engine yet, set it up like this
##### Session = sessionmaker() 
Later when you create your engine, connect it to the sessions using
###### Session.configure(bind=engine)

# Adding and Updating Objects

In [14]:
session = Session()
session.add(ed_user)

In [15]:
our_user = session.query(User).filter_by(name='ed').first()

2022-09-14 02:11:25,250 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 02:11:25,305 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-09-14 02:11:25,309 INFO sqlalchemy.engine.Engine [generated in 0.00433s] ('ed', 'Ed Jones', 'edsnickname')
2022-09-14 02:11:25,354 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2022-09-14 02:11:25,356 INFO sqlalchemy.engine.Engine [generated in 0.00272s] ('ed', 1, 0)


In [16]:
print(our_user)

<User(name=ed, fullname=Ed Jones, nickname=edsnickname)


In [17]:
ed_user is our_user

True

In [18]:
session.add_all([
    User(name="wendy", fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')
])

In [19]:
ed_user.nickname = 'eddie'

In [20]:
session.dirty

IdentitySet([<User(name=ed, fullname=Ed Jones, nickname=eddie)])

In [21]:
session.new

IdentitySet([<User(name=wendy, fullname=Wendy Williams, nickname=windy), <User(name=mary, fullname=Mary Contrary, nickname=mary), <User(name=fred, fullname=Fred Flintstone, nickname=freddy)])

In [22]:
session.commit()

2022-09-14 02:11:26,490 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?
2022-09-14 02:11:26,491 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ('eddie', 1)
2022-09-14 02:11:26,493 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-09-14 02:11:26,494 INFO sqlalchemy.engine.Engine [cached since 1.19s ago] ('wendy', 'Wendy Williams', 'windy')
2022-09-14 02:11:26,496 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-09-14 02:11:26,497 INFO sqlalchemy.engine.Engine [cached since 1.193s ago] ('mary', 'Mary Contrary', 'mary')
2022-09-14 02:11:26,501 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-09-14 02:11:26,502 INFO sqlalchemy.engine.Engine [cached since 1.198s ago] ('fred', 'Fred Flintstone', 'freddy')
2022-09-14 02:11:26,505 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
ed_user.id

2022-09-14 02:11:26,678 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 02:11:26,681 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2022-09-14 02:11:26,682 INFO sqlalchemy.engine.Engine [generated in 0.00136s] (1,)


1

In [24]:
ed_user.name = 'Edwardo'
fake_user = User(name='fakeuser', fullname='Invalid',nickname='12345')
session.add(fake_user)

In [25]:
session.new

IdentitySet([<User(name=fakeuser, fullname=Invalid, nickname=12345)])

In [26]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2022-09-14 02:11:27,147 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2022-09-14 02:11:27,149 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ('Edwardo', 1)
2022-09-14 02:11:27,152 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-09-14 02:11:27,153 INFO sqlalchemy.engine.Engine [cached since 1.848s ago] ('fakeuser', 'Invalid', '12345')
2022-09-14 02:11:27,157 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2022-09-14 02:11:27,158 INFO sqlalchemy.engine.Engine [generated in 0.00240s] ('Edwardo', 'fakeuser')


[<User(name=Edwardo, fullname=Ed Jones, nickname=eddie),
 <User(name=fakeuser, fullname=Invalid, nickname=12345)]

In [27]:
session.rollback()

2022-09-14 02:11:27,303 INFO sqlalchemy.engine.Engine ROLLBACK


In [28]:
ed_user.name

2022-09-14 02:11:27,511 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-14 02:11:27,513 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2022-09-14 02:11:27,515 INFO sqlalchemy.engine.Engine [cached since 0.8338s ago] (1,)


'ed'

In [29]:
fake_user in session

False

# Querying

In [30]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2022-09-14 02:11:28,669 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2022-09-14 02:11:28,671 INFO sqlalchemy.engine.Engine [generated in 0.00212s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [31]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2022-09-14 02:11:29,197 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2022-09-14 02:11:29,199 INFO sqlalchemy.engine.Engine [generated in 0.00202s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [32]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2022-09-14 02:11:29,626 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.name AS users_name__1 
FROM users
2022-09-14 02:11:29,628 INFO sqlalchemy.engine.Engine [generated in 0.00220s] ()
<User(name=ed, fullname=Ed Jones, nickname=eddie) ed
<User(name=wendy, fullname=Wendy Williams, nickname=windy) wendy
<User(name=mary, fullname=Mary Contrary, nickname=mary) mary
<User(name=fred, fullname=Fred Flintstone, nickname=freddy) fred


In [33]:
for row in session.query(User):
    print(row)

2022-09-14 02:11:30,054 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2022-09-14 02:11:30,056 INFO sqlalchemy.engine.Engine [generated in 0.00214s] ()
<User(name=ed, fullname=Ed Jones, nickname=eddie)
<User(name=wendy, fullname=Wendy Williams, nickname=windy)
<User(name=mary, fullname=Mary Contrary, nickname=mary)
<User(name=fred, fullname=Fred Flintstone, nickname=freddy)


In [34]:
for row in  session.query(User.name.label('name_label')).all():
    print(row.name_label)

2022-09-14 02:11:30,464 INFO sqlalchemy.engine.Engine SELECT users.name AS name_label 
FROM users
2022-09-14 02:11:30,466 INFO sqlalchemy.engine.Engine [generated in 0.00199s] ()
ed
wendy
mary
fred


In [35]:
user_alias = aliased(User, name='alias')
for row in session.query(user_alias, user_alias.name).all():
    print(row.alias)

2022-09-14 02:11:30,859 INFO sqlalchemy.engine.Engine SELECT alias.id AS alias_id, alias.name AS alias_name, alias.fullname AS alias_fullname, alias.nickname AS alias_nickname, alias.name AS alias_name__1 
FROM users AS alias
2022-09-14 02:11:30,861 INFO sqlalchemy.engine.Engine [generated in 0.00207s] ()
<User(name=ed, fullname=Ed Jones, nickname=eddie)
<User(name=wendy, fullname=Wendy Williams, nickname=windy)
<User(name=mary, fullname=Mary Contrary, nickname=mary)
<User(name=fred, fullname=Fred Flintstone, nickname=freddy)


In [36]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2022-09-14 02:11:31,235 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2022-09-14 02:11:31,237 INFO sqlalchemy.engine.Engine [generated in 0.00195s] (2, 1)
<User(name=wendy, fullname=Wendy Williams, nickname=windy)
<User(name=mary, fullname=Mary Contrary, nickname=mary)


In [37]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2022-09-14 02:11:31,612 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2022-09-14 02:11:31,615 INFO sqlalchemy.engine.Engine [generated in 0.00222s] ('Ed Jones',)
ed


In [38]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2022-09-14 02:11:31,961 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2022-09-14 02:11:31,963 INFO sqlalchemy.engine.Engine [cached since 0.3505s ago] ('Ed Jones',)
ed


In [39]:
for name in session.query(User).filter_by(id = 1).filter_by(name='mary'):
    print(name)

2022-09-14 02:11:32,313 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ? AND users.name = ?
2022-09-14 02:11:32,315 INFO sqlalchemy.engine.Engine [generated in 0.00205s] (1, 'mary')


In [40]:
for user in session.query(User).\
         filter(User.name=='ed').\
         filter(User.fullname=='Ed Jones'):
    print(user)

2022-09-14 02:11:32,650 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2022-09-14 02:11:32,652 INFO sqlalchemy.engine.Engine [generated in 0.00215s] ('ed', 'Ed Jones')
<User(name=ed, fullname=Ed Jones, nickname=eddie)


###### Filtering with different and common filter operator


In [41]:
jame = session.query(User).filter(User.name!='ed')

In [42]:
jame.all()

2022-09-14 02:11:33,606 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name != ?
2022-09-14 02:11:33,608 INFO sqlalchemy.engine.Engine [generated in 0.00176s] ('ed',)


[<User(name=wendy, fullname=Wendy Williams, nickname=windy),
 <User(name=mary, fullname=Mary Contrary, nickname=mary),
 <User(name=fred, fullname=Fred Flintstone, nickname=freddy)]

In [43]:
for i in session.query(User).filter(User.name.like('%mary%')):
    print(i.name)

2022-09-14 02:11:33,917 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ?
2022-09-14 02:11:33,918 INFO sqlalchemy.engine.Engine [generated in 0.00187s] ('%mary%',)
mary


In [44]:
# Filtering with ilike which is used to compare rows to
# see if a certain string in the function argument is in
# the query
for j in session.query(User).filter(User.name.ilike('%e%')):
    print(j.name)

2022-09-14 02:11:34,245 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE lower(users.name) LIKE lower(?)
2022-09-14 02:11:34,248 INFO sqlalchemy.engine.Engine [generated in 0.00252s] ('%e%',)
ed
wendy
fred


In [45]:
# Filtering with a normal List
print("Filtering with a link")
for link in session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])):
    print(link)

# Filtering with an in_ query inside another query
print("Filtering with in_ query inside a query")
for i in session.query(User).filter(
    User.name.in_(
        session.query(User.name).filter(User.name.like('%e%'))
    )
):
    print(i.name)
    
# Filtering with a tuple_() for composite (Multi-column) queries
print("Filtering with a tuple_() function for composite columns")
for tup in session.query(User).filter(
    tuple_(User.name, User.nickname).\
    in_([('ed', 'edsnickname'), ('wendy', 'windy')])
):
    print(tup)

Filtering with a link
2022-09-14 02:11:34,577 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?, ?)
2022-09-14 02:11:34,578 INFO sqlalchemy.engine.Engine [cached since 7.423s ago] ('ed', 'wendy', 'jack')
<User(name=ed, fullname=Ed Jones, nickname=eddie)
<User(name=wendy, fullname=Wendy Williams, nickname=windy)
Filtering with in_ query inside a query
2022-09-14 02:11:34,606 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (SELECT users.name 
FROM users 
WHERE users.name LIKE ?)
2022-09-14 02:11:34,609 INFO sqlalchemy.engine.Engine [generated in 0.00234s] ('%e%',)
ed
wendy
fred
Filtering with a tuple_() function for composite columns
2022-09-14 02:11:34,671 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id,

In [46]:
# Using the notin function to filter a query
for i in session.query(User).filter(~User.name.in_(['ed', 'wendy','jack'])):
    print(i)

2022-09-14 02:11:34,962 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE (users.name NOT IN (?, ?, ?))
2022-09-14 02:11:34,964 INFO sqlalchemy.engine.Engine [generated in 0.00235s] ('ed', 'wendy', 'jack')
<User(name=mary, fullname=Mary Contrary, nickname=mary)
<User(name=fred, fullname=Fred Flintstone, nickname=freddy)


In [47]:
# Filtering using the is_ function
jame = session.query(User).filter(User.name==None)
print("Using the == sign for is")
print(jame.all())
jame = session.query(User).filter(User.name.is_(None))
print("Using the is_ sign now")
print(jame.all())

Using the == sign for is
2022-09-14 02:11:35,422 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IS NULL
2022-09-14 02:11:35,424 INFO sqlalchemy.engine.Engine [generated in 0.00194s] ()
[]
Using the is_ sign now
2022-09-14 02:11:35,428 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IS NULL
2022-09-14 02:11:35,430 INFO sqlalchemy.engine.Engine [cached since 0.008007s ago] ()
[]


In [48]:
# Filtering using the isnot function
jame = session.query(User).filter(User.name!=None)
print("Using the != sign for isnot")
print(jame.all())
jame = session.query(User).filter(User.name.isnot(None))
print("Using the isnot sign now")
print(jame.all())

Using the != sign for isnot
2022-09-14 02:11:35,971 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IS NOT NULL
2022-09-14 02:11:35,972 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ()
[<User(name=ed, fullname=Ed Jones, nickname=eddie), <User(name=wendy, fullname=Wendy Williams, nickname=windy), <User(name=mary, fullname=Mary Contrary, nickname=mary), <User(name=fred, fullname=Fred Flintstone, nickname=freddy)]
Using the isnot sign now
2022-09-14 02:11:35,978 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IS NOT NULL
2022-09-14 02:11:35,980 INFO sqlalchemy.engine.Engine [cached since 0.009736s ago] ()
[<User(name=ed, fullname=Ed Jones, nickname=eddie), <User(name=wendy, fullname=Wendy Williams, nickname=windy), <U

In [49]:
# Filtering using the and_ command
jame = session.query(User).filter(and_(User.name=='ed', User.fullname=='Ed Jones'))
# Filtering by sending multiple expressions to the .filter function
kame = session.query(User).filter(User.name=='ed', User.fullname=='Ed Jones')
# Filtering by chaining multiple filter/filter_by functions
lame = session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones')
print(jame.all())
print(kame.all())
print(lame.all())

2022-09-14 02:11:36,502 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2022-09-14 02:11:36,504 INFO sqlalchemy.engine.Engine [generated in 0.00201s] ('ed', 'Ed Jones')
[<User(name=ed, fullname=Ed Jones, nickname=eddie)]
2022-09-14 02:11:36,509 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2022-09-14 02:11:36,510 INFO sqlalchemy.engine.Engine [cached since 3.86s ago] ('ed', 'Ed Jones')
[<User(name=ed, fullname=Ed Jones, nickname=eddie)]
2022-09-14 02:11:36,513 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
20

In [50]:
# Filtering using the sqlalchemy or function
jame = session.query(User).filter(or_(User.name=='ed', User.name=='Wendy'))
jame.all()

2022-09-14 02:11:37,097 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? OR users.name = ?
2022-09-14 02:11:37,099 INFO sqlalchemy.engine.Engine [generated in 0.00189s] ('ed', 'Wendy')


[<User(name=ed, fullname=Ed Jones, nickname=eddie)]

In [51]:
"""
    Filtering using the match function
    Match is used to display queries which contains the 
    arguments in the match function
"""
# jame = session.query(User).filter(User.name.match('wendy'))
# jame.all()

# This is not surpported in SQLite so it would not work

'\n    Filtering using the match function\n    Match is used to display queries which contains the \n    arguments in the match function\n'

In [54]:
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('User', back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    

In [55]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")